In [ ]:
import Levenshtein
def pair2edits_new(src, trg):
    src_line = ''.join(src.strip().replace(',', '，').split())
    tgt_line = ''.join(trg.strip().replace(',', '，').split())
    # sid = lines_sid[i].strip().split('\t')[0]

    # edits = Levenshtein.opcodes(src_line, tgt_line)
    # reverse
    _edits = Levenshtein.opcodes(src_line[::-1], tgt_line[::-1])[::-1]
    edits = []
    src_len = len(src_line)
    tgt_len = len(tgt_line)
    for edit in _edits:
        edits.append((edit[0], src_len - edit[2], src_len - edit[1], tgt_len - edit[4], tgt_len - edit[3]))

    # merge coterminous Levenshtein edited spans
    merged_edits = []
    for edit in edits:
        if edit[0] == 'equal':
            continue
        if len(merged_edits) > 0:
            last_edit = merged_edits[-1]
            if last_edit[0] == 'insert' and edit[0] == 'insert' and last_edit[2] == edit[1]:
                new_edit = ('insert', last_edit[1], edit[2], last_edit[3], edit[4])
                merged_edits[-1] = new_edit
            elif last_edit[2] == edit[1]:
                assert last_edit[4] == edit[3]
                new_edit = ('hybrid', last_edit[1], edit[2], last_edit[3], edit[4])
                merged_edits[-1] = new_edit
            elif last_edit[0] == 'insert' and edit[0] == 'delete' \
                and tgt_line[last_edit[3]:last_edit[4]] == src_line[edit[1]:edit[2]]:
                new_edit = ('luanxu', last_edit[1], edit[2], last_edit[3], edit[4])
                merged_edits[-1] = new_edit
            elif last_edit[0] == 'delete' and edit[0] == 'insert':
                if src_line[last_edit[1]:last_edit[2]] == tgt_line[edit[3]:edit[4]]:
                # print(src_line[last_edit[1]:last_edit[2]] == tgt_line[edit[3]:edit[4]])
                    new_edit = ('luanxu', last_edit[1], edit[2], last_edit[3], edit[4])
                    merged_edits[-1] = new_edit
                elif edit[4] < len(tgt_line) and tgt_line[edit[3]] == tgt_line[edit[4]] and src_line[last_edit[1]:last_edit[2]] == tgt_line[edit[3]+1:edit[4]+1]:
                    new_edit = ('luanxu', last_edit[1], edit[2]+1, last_edit[3], edit[4])
                    merged_edits[-1] = new_edit
                else:
                    merged_edits.append(edit)
            else:
                merged_edits.append(edit)
        else:
            merged_edits.append(edit)
    merged_edits2 = []
    for edit in merged_edits:
        if edit[0] == 'equal':
            continue
        if len(merged_edits2) > 0:
            last_edit = merged_edits2[-1]
            if last_edit[0] == 'insert' and edit[0] == 'insert' and last_edit[2] == edit[1]:
                new_edit = ('insert', last_edit[1], edit[2], last_edit[3], edit[4])
                merged_edits2[-1] = new_edit
            elif last_edit[2] == edit[1]:
                assert last_edit[4] == edit[3]
                new_edit = ('hybrid', last_edit[1], edit[2], last_edit[3], edit[4])
                merged_edits2[-1] = new_edit
            elif last_edit[0] == 'insert' and edit[0] == 'delete' \
                and tgt_line[last_edit[3]:last_edit[4]] == src_line[edit[1]:edit[2]]:
                new_edit = ('luanxu', last_edit[1], edit[2], last_edit[3], edit[4])
                merged_edits2[-1] = new_edit
            elif last_edit[0] == 'delete' and edit[0] == 'insert':
                if src_line[last_edit[1]:last_edit[2]] == tgt_line[edit[3]:edit[4]]:
                # print(src_line[last_edit[1]:last_edit[2]] == tgt_line[edit[3]:edit[4]])
                    new_edit = ('luanxu', last_edit[1], edit[2], last_edit[3], edit[4])
                    merged_edits2[-1] = new_edit
                elif edit[4] < len(tgt_line) and tgt_line[edit[3]] == tgt_line[edit[4]] and src_line[last_edit[1]:last_edit[2]] == tgt_line[edit[3]+1:edit[4]+1]:
                    new_edit = ('luanxu', last_edit[1], edit[2]+1, last_edit[3], edit[4])
                    merged_edits2[-1] = new_edit
                else:
                    merged_edits2.append(edit)
            else:
                merged_edits2.append(edit)
        else:
            merged_edits2.append(edit)
    # generate edit sequence
    result = []
    for edit in merged_edits2:
        if tgt_line[edit[3]:edit[4]] == '[UNK]':
            continue
        if edit[0] == "insert":
            result.append((str(edit[1]+1), str(edit[1]+1), "M", tgt_line[edit[3]:edit[4]]))
        elif edit[0] == "replace":
            # new_op = post_process_S(src_line, (str(edit[1]+1), str(edit[2]), "S", tgt_line[edit[3]:edit[4]]))
            result.append((str(edit[1]+1), str(edit[2]), "S", tgt_line[edit[3]:edit[4]]))
        elif edit[0] == "delete":
            result.append((str(edit[1]+1), str(edit[2]), "R"))
        elif edit[0] == "hybrid":
            # new_op = post_process_S(src_line, (str(edit[1]+1), str(edit[2]), "S", tgt_line[edit[3]:edit[4]]))
            result.append((str(edit[1]+1), str(edit[2]), "S", tgt_line[edit[3]:edit[4]]))
        elif edit[0] == "luanxu":
            result.append((str(edit[1]+1), str(edit[2]) , "W"))

    # print
    # out_line = id +',\t'
    return result

In [ ]:
import json
data = open('/home/liyunliang/CGED_Task/dataset/lang8/lang8_data_v1.jsonl').readlines()
print(len(data))
print(data[0])

In [ ]:
# 使用新版本的编辑转换文件生成错误点
cnt = 0
new_data = []
for i, line in enumerate(data):
    line = json.loads(line)
    errors = []
    assert line['correct'] != ''
        # cnt += 1
    try:
        edits = pair2edits_new(line['text']+'。', line['correct']+'。')
    except:
        continue
    if edits == []:
        assert line['error'] == []
        new_data.append(json.dumps(line, ensure_ascii=False))
        cnt += 1
        continue
    for edit in edits:
        assert len(edit) in [3, 4] and int(edit[1]) >= int(edit[0])
        # errors.append({'start': int(edit[0])-1, 'end': int(edit[1]), 'type': edit[2]})
        if len(edit) == 4:
            errors.append({'start': int(edit[0])-1, 'end': int(edit[1]), 'type': edit[2], 'correct': edit[3]})
        else:
            errors.append({'start': int(edit[0])-1, 'end': int(edit[1]), 'type': edit[2], 'correct': ''})
    # line['error'] = errors[:]
    # data[i] = json.dumps(line, ensure_ascii=False)
    try:
        assert errors == line['error']
        # line['error'] = errors[:]
        new_data.append(json.dumps(line, ensure_ascii=False))
        cnt += 1
    except:
        # print(line['text'], ' | ', line['correct'])
        # print(line['error'], ' | ', pair2edits(line['text']+'。', line['correct']+'。'))
        # print(errors)
        # print()
        flag = 0
        for error in errors:
            if error['start'] >= len(line['text']) or error['end'] > len(line['text']):
                flag = 1
                break
        if flag:
            continue
        line['error'] = errors[:]
        new_data.append(json.dumps(line, ensure_ascii=False))
        cnt += 1

    
    # else:
    #     data[i] = data[i].strip()
print(cnt)
open('lang8_data_v2.jsonl', 'w').write('\n'.join(new_data))